In [1]:
from colossus.cosmology import cosmology
cosmo=cosmology.setCosmology('planck18')
import numpy as np
import Func
from cobaya.run import run
from getdist import loadMCSamples, plots

# data and fid

In [2]:
zz = np.array([0.15,0.45,0.75])
# error of (f*sigma8) at zz
err_rsd = np.array([0.037622781639503125,0.013120288894571081,0.00801293154199452])
err_rsd_ksz = np.array([0.0048570460593981195,0.0029804102835748773,0.0028959660223023967])

In [3]:
# Fiducial values  
w_fid = -1.58491121
omegam_fid = 0.19765297
sigma8_fid = 0.97234554

fs8 = Func.growthrate(Om0=omegam_fid,w=w_fid,z=zz) * Func.sigma(sigma8=sigma8_fid, Om0=omegam_fid,w=w_fid,z=zz)
ref=[omegam_fid,sigma8_fid,w_fid]

# RSD

In [4]:
def da(z_array, omegam, w):
    results = []
    for z in z_array:
        int_z = np.linspace(0, z, 50)
        E = Func.Ez(int_z, omegam, w)
        integral = np.trapz(1./E, int_z)
        results.append(integral)
    return np.array(results)

def log_like1(omegam,sigma8,w):

    E = Func.Ez(zz, omegam, w)
    E_fid = Func.Ez(zz, omegam_fid, w_fid)

    D = da(zz, omegam, w)
    D_fid = da(zz, omegam_fid, w_fid)

    f_th = Func.growthrate(Om0=omegam,w=w, z=zz)
    sigma8_th = Func.sigma(sigma8=sigma8, Om0=omegam, w=w, z=zz)
    
    Chi_1 = (fs8 - (E*D)/(E_fid*D_fid)*sigma8_th*f_th)

    Chi_2 = err_rsd**2
    Chi_C = Chi_1**2/Chi_2
    Chi = np.sum(Chi_C)
    
    return -0.5*Chi


info = {"likelihood": {"loglike": log_like1}, \
        "params": {"omegam": {"prior": {"min": 0.0, "max": 1.0},'ref': ref[0],"latex": r'\Omega_m'}, \
                    "sigma8": {"prior": {"min": 0.5, "max": 1.5},'ref': ref[1],"latex": r'\sigma8'}, \
                    "w": {"prior": {"min": -2., "max": 0.0},'ref': ref[2],"latex": r'w'}
                    }, \
        "sampler": {"mcmc": {"Rminus1_stop": 0.01, "max_tries": 10000},},\
        "output": "chains_rsd/rsd"
        }
# ,"proposal": 0.001
updated_info, sampler = run(info,force=True)

[output] Output to be read-from/written-into folder 'chains_rsd', with prefix 'rsd'
[output] Found existing info files with the requested output prefix: 'chains_rsd/rsd'
[output] Will delete previous products ('force' was requested).
[loglike] Initialized external likelihood.
[mcmc] Getting initial point... (this may take a few seconds)
[mcmc] Initial point: omegam:0.197653, sigma8:0.9723455, w:-1.584911
[model] Measuring speeds... (this may take a few seconds)
[model] Setting measured speeds (per sec): {loglike: 312.0}
[prior] *WARNING* Reference pdf not defined or improper for some parameters. Using prior's sigma instead for them.
[mcmc] Covariance matrix not present. We will start learning the covariance of the proposal earlier: R-1 = 30 (would be 2 if all params loaded).
[mcmc] Sampling!
[mcmc] Progress @ 2024-05-18 11:40:55 : 1 steps taken, and 0 accepted.
[mcmc] Learn + convergence test @ 120 samples accepted.
[mcmc]  - Acceptance rate: 0.039
[mcmc]  - Convergence of means: R-1 =

In [5]:
readsamps1 = loadMCSamples('chains_rsd/rsd')
cov_rsd = readsamps1.cov(['w','omegam','sigma8'])
print("cov is:")
print(cov_rsd)
fisher_rsd = np.linalg.inv(cov_rsd)
print("fisher is:")
np.array(fisher_rsd)

cov is:
[[ 0.25703697 -0.00260834  0.0636497 ]
 [-0.00260834  0.00188359 -0.00386036]
 [ 0.0636497  -0.00386036  0.02334124]]
fisher is:


array([[  30.20200579, -192.07318699, -114.12502183],
       [-192.07318699, 2024.64107128,  858.61920149],
       [-114.12502183,  858.61920149,  496.05759245]])

# RSD and kSZ

In [8]:
def log_like2(omegam,sigma8,w):

    E = Func.Ez(zz, omegam, w)
    E_fid = Func.Ez(zz, omegam_fid, w_fid)

    D = da(zz, omegam, w)
    D_fid = da(zz, omegam_fid, w_fid)

    f_th = Func.growthrate(Om0=omegam,w=w, z=zz)
    sigma8_th = Func.sigma(sigma8=sigma8, Om0=omegam, w=w, z=zz)
    
    Chi_1 = (fs8 - (E*D)/(E_fid*D_fid)*sigma8_th*f_th)

    Chi_2 = err_rsd_ksz**2
    Chi_C = Chi_1**2/Chi_2
    Chi = np.sum(Chi_C)
    
    return -0.5*Chi


info = {"likelihood": {"loglike": log_like2}, \
        "params": {"omegam": {"prior": {"min": 0.0, "max": 1.0},'ref': ref[0],"latex": r'\Omega_m'}, \
                    "sigma8": {"prior": {"min": 0.5, "max": 1.5},'ref': ref[1],"latex": r'\sigma8'}, \
                    "w": {"prior": {"min": -2., "max": 0.0},'ref': ref[2],"latex": r'w'}
                    }, \
        "sampler": {"mcmc": {"Rminus1_stop": 0.01, "max_tries": 10000},},\
        "output": "chains_rsd_ksz/rsd_ksz"
        }
# ,"proposal": 0.001
updated_info, sampler = run(info,force=True)

[output] Output to be read-from/written-into folder 'chains_rsd_ksz', with prefix 'rsd_ksz'
[output] Found existing info files with the requested output prefix: 'chains_rsd_ksz/rsd_ksz'
[output] Will delete previous products ('force' was requested).
[loglike] Initialized external likelihood.
[mcmc] Getting initial point... (this may take a few seconds)
[mcmc] Initial point: omegam:0.197653, sigma8:0.9723455, w:-1.584911
[model] Measuring speeds... (this may take a few seconds)
[model] Setting measured speeds (per sec): {loglike: 127.0}
[prior] *WARNING* Reference pdf not defined or improper for some parameters. Using prior's sigma instead for them.
[mcmc] Covariance matrix not present. We will start learning the covariance of the proposal earlier: R-1 = 30 (would be 2 if all params loaded).
[mcmc] Sampling!
[mcmc] Progress @ 2024-05-18 11:43:34 : 1 steps taken, and 0 accepted.
[mcmc] Learn + convergence test @ 120 samples accepted.
[mcmc]  - Acceptance rate: 0.010
[mcmc]  - Convergence

In [9]:
readsamps2 = loadMCSamples('chains_rsd_ksz/rsd_ksz')
cov_rsd_ksz = readsamps2.cov(['w','omegam','sigma8'])
print("cov is:")
print(cov_rsd_ksz)
fisher_rsd_ksz = np.linalg.inv(cov_rsd_ksz)
print("fisher is:")
np.array(fisher_rsd_ksz)

[root] *WARNING* outlier fraction 0.01989247311827957 
cov is:
[[ 2.72066790e-02 -5.13513837e-05  3.68896704e-03]
 [-5.13513837e-05  5.71459513e-05 -8.84819454e-05]
 [ 3.68896704e-03 -8.84819454e-05  6.32914982e-04]]
fisher is:


array([[  1124.24815622, -11659.46790013,  -8182.72112514],
       [-11659.46790013, 143252.53907746,  87984.41764765],
       [ -8182.72112514,  87984.41764765,  61573.54789072]])

# joint

In [10]:
g=plots.get_subplot_plotter(chain_dir=['/home/wangsy/mcmc/wcdm/chains_rsd',
                                       '/home/wangsy/mcmc/wcdm/chains_rsd_ksz',
                                       '/home/wangsy/chains_planck/base_w/plikHM_TTTEEE_lowl_lowE'
                                       ],
                                       width_inch=7)
roots = [ 'rsd','rsd_ksz','base_w_plikHM_TTTEEE_lowl_lowE']
params = ['w','omegam','sigma8']
g.triangle_plot(roots, params, filled=True, legend_labels=['RSD','RSD+kSZ tomography', 'Planck'])
g.export('wcdm1.pdf')
